In [13]:
%reload_ext dotenv
%dotenv

In [19]:
import ocean_lib
import sys
print(f"Using python:{sys.version}")
print(f"Using ocean_lib: {ocean_lib.__version__}")

Using python:3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) 
[GCC 9.3.0]
Using ocean_lib: 0.8.5


# 1. Create ocean instance

In [6]:
from ocean_lib.config import Config
from ocean_lib.ocean.ocean import Ocean
config = Config('./config-dataunion-mumbai.ini')
ocean = Ocean(config)

print(f"config.network_url = '{config.network_url}'")
print(f"config.block_confirmations = {config.block_confirmations.value}")
print(f"config.metadata_cache_uri = '{config.metadata_cache_uri}'")
print(f"config.provider_url = '{config.provider_url}'")

# Create Alice's wallet
import os
from ocean_lib.web3_internal.wallet import Wallet
alice_wallet = Wallet(
    ocean.web3,
    os.getenv('TEST_PRIVATE_KEY1'),
    config.block_confirmations,
    config.transaction_timeout,
)
print(f"alice_wallet.address = '{alice_wallet.address}'")

config.network_url = 'https://rpc-mumbai.maticvigil.com'
config.block_confirmations = 1
config.metadata_cache_uri = 'https://aquarius.dataunion.market'
config.provider_url = 'https://provider.dev.dataunion.market/'
alice_wallet.address = '0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA'


# 2. Publish asset

In [7]:
# Publish DATA datatoken, mint tokens
from ocean_lib.web3_internal.currency import to_wei

DATA_datatoken = ocean.create_data_token('DATA1', 'DATA1', alice_wallet, blob=ocean.config.metadata_cache_uri)
DATA_datatoken.mint(alice_wallet.address, to_wei(100), alice_wallet)
print(f"DATA_datatoken.address = '{DATA_datatoken.address}'")

# Specify metadata & service attributes for Branin test dataset.
# It's specified using _local_ DDO metadata format; Aquarius will convert it to remote
# by removing `url` and adding `encryptedFiles` field.
DATA_metadata = {
    "main": {
        "type": "dataset",
        "files": [
      {
        "url": "https://raw.githubusercontent.com/trentmc/branin/main/branin.arff",
        "index": 0,
        "contentType": "text/text"
      }
    ],
    "name": "branin", "author": "Trent", "license": "CC0",
    "dateCreated": "2019-12-28T10:55:11Z"
    }
}
DATA_service_attributes = {
    "main": {
        "name": "DATA_dataAssetAccessServiceAgreement",
        "creator": alice_wallet.address,
        "timeout": 3600 * 24,
        "datePublished": "2019-12-28T10:55:11Z",
        "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Set up a service provider. We'll use this same provider for ALG
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
provider_url = DataServiceProvider.get_url(ocean.config)


# Calc DATA service compute descriptor
from ocean_lib.services.service import Service
from ocean_lib.common.agreements.service_types import ServiceTypes
DATA_compute_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=DATA_service_attributes
)

# Publish metadata and service info on-chain
DATA_ddo = ocean.assets.create(
  metadata=DATA_metadata, # {"main" : {"type" : "dataset", ..}, ..}
  publisher_wallet=alice_wallet,
  services=[DATA_compute_service],
  data_token_address=DATA_datatoken.address)
print(f"DATA did = '{DATA_ddo.did}'")

DATA_datatoken.address = '0x89eF6B9eb899B6C86dCFBBa0E5A96997506A304b'


INFO:ocean_lib.data_provider.data_service_provider:Asset urls encrypted successfully, encrypted urls str: {"encryptedDocument": "0x041a203809ddb23d6ef31f0fbae39f5e12eae5d54d8ce0ba6e975722cc4c43afa1071233b73d629a0641229f29a6225511585ef91fde2f5aabe15928079cf5d41fad32dc3ecc4d2dda758467deb29b892fddb3c6b3700748ac413d228e62c3257869fd1c5c29a3bcca6940c6f663f385c6a33c6d05c1843c089ace8a0adc306aa838a2228d13b51620707bf4c2b5ad80c847619603b48f01851e2f77ececed0375d1b99dc0e937aba73ce720d745fb8805fc69a6ea7971fbf10dbe68d138b0221cbb0544075a282a7ed7ca3debaf5fd9160d"}, encryptedEndpoint https://provider.dev.dataunion.market/api/v1/services/encrypt


signing message with nonce 7: 0x89eF6B9eb899B6C86dCFBBa0E5A96997506A304b, account=0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA


INFO:ocean:Asset/ddo published on-chain successfully.


DATA did = 'did:op:89eF6B9eb899B6C86dCFBBa0E5A96997506A304b'


In [52]:
# Publish DATA datatoken, mint tokens
from ocean_lib.web3_internal.currency import to_wei

DATA_datatoken = ocean.create_data_token('DATA1', 'DATA1', publisher_wallet, blob=ocean.config.metadata_cache_uri)
DATA_datatoken.mint(publisher_wallet.address, to_wei(100), publisher_wallet)
print(f"DATA_datatoken.address = '{DATA_datatoken.address}'")

# Specify metadata & service attributes for Branin test dataset.
# It's specified using _local_ DDO metadata format; Aquarius will convert it to remote
# by removing `url` and adding `encryptedFiles` field.
DATA_metadata = {
    "main": {
        "type": "dataset",
        "files": [
      {
        "url": "https://raw.githubusercontent.com/trentmc/branin/main/branin.arff",
        "index": 0,
        "contentType": "text/text"
      }
    ],
    "name": "branin", "author": "Trent", "license": "CC0",
    "dateCreated": "2019-12-28T10:55:11Z"
    }
}
DATA_service_attributes = {
    "main": {
        "name": "DATA_dataAssetAccessServiceAgreement",
        "creator": publisher_wallet.address,
        "timeout": 3600 * 24,
        "datePublished": "2019-12-28T10:55:11Z",
        "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Set up a service provider. We'll use this same provider for ALG
from ocean_lib.data_provider.data_service_provider import DataServiceProvider
provider_url = DataServiceProvider.get_url(ocean.config)
# returns "http://localhost:8030"

# Calc DATA service compute descriptor
from ocean_lib.services.service import Service
from ocean_lib.common.agreements.service_types import ServiceTypes
DATA_compute_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=DATA_service_attributes
)

#Publish metadata and service info on-chain
DATA_ddo = ocean.assets.create(
  metadata=DATA_metadata, # {"main" : {"type" : "dataset", ..}, ..}
  publisher_wallet=publisher_wallet,
  services=[DATA_compute_service],
  data_token_address=DATA_datatoken.address)
print(f"DATA did = '{DATA_ddo.did}'")

DATA_datatoken.address = '0xd3b8e939303D9af0BDb7774579D2fA8d441be2E9'
signing message with nonce 4: 0xd3b8e939303D9af0BDb7774579D2fA8d441be2E9, account=0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA
DATA did = 'did:op:d3b8e939303D9af0BDb7774579D2fA8d441be2E9'


# 3. Publish algorithm

In [8]:
# Publish ALG datatoken
ALG_datatoken = ocean.create_data_token('ALG1', 'ALG1', alice_wallet, blob=ocean.config.metadata_cache_uri)
ALG_datatoken.mint(alice_wallet.address, to_wei(100), alice_wallet)
print(f"ALG_datatoken.address = '{ALG_datatoken.address}'")

# Specify metadata and service attributes, for "GPR" algorithm script.
# In same location as Branin test dataset. GPR = Gaussian Process Regression.
ALG_metadata =  {
    "main": {
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "docker-image",
            "version": "0.1",
            "container": {
              "entrypoint": "python $ALGO",
              "image": "oceanprotocol/algo_dockers",
              "tag": "python-branin"
            }
        },
        "files": [
      {
        "url": "https://raw.githubusercontent.com/trentmc/branin/main/gpr.py",
        "index": 0,
        "contentType": "text/text",
      }
    ],
    "name": "gpr", "author": "Trent", "license": "CC0",
    "dateCreated": "2020-01-28T10:55:11Z"
    }
}
ALG_service_attributes = {
        "main": {
            "name": "ALG_dataAssetAccessServiceAgreement",
            "creator": alice_wallet.address,
            "timeout": 3600 * 24,
            "datePublished": "2020-01-28T10:55:11Z",
            "cost": 1.0, # <don't change, this is obsolete>
        }
    }

# Calc ALG service access descriptor. We use the same service provider as DATA
ALG_access_service = Service(
    service_endpoint=provider_url,
    service_type=ServiceTypes.CLOUD_COMPUTE,
    attributes=ALG_service_attributes
)

# Publish metadata and service info on-chain
ALG_ddo = ocean.assets.create(
  metadata=ALG_metadata, # {"main" : {"type" : "algorithm", ..}, ..}
  publisher_wallet=alice_wallet,
  services=[ALG_access_service],
  data_token_address=ALG_datatoken.address)
print(f"ALG did = '{ALG_ddo.did}'")

ALG_datatoken.address = '0x8C96958d4efd521f586829B3f8ede01B83E3478C'


INFO:ocean_lib.data_provider.data_service_provider:Asset urls encrypted successfully, encrypted urls str: {"encryptedDocument": "0x04b628784b6e60c5b19bdd511f7576939a035d5c9c761e89b54ba34618af65f9ce7a6eb2924c3a27d1c841a851b64e0c19ce501d3e49f1151c960859e4dcf2448280205effd5b3f17b1145b6401a838978fa62b58f99425b1353ba59f9c30943cdf4eddf21bccafec9a3b4d5382307ba556c960b41d402e1f7c0746c004b7e1213cc441759c44ba5234a3551a370b2ecb2198143a391a7fc99f90ec44b13c7823a967a8c8dd50d782af2e331b9d54ac1d31f2a3121ae8f6022cd1aa3cb95a1b7c67a2400e1644cfdcba86de918"}, encryptedEndpoint https://provider.dev.dataunion.market/api/v1/services/encrypt


signing message with nonce 8: 0x8C96958d4efd521f586829B3f8ede01B83E3478C, account=0xdF1dEc52e602020E27B0644Ea0F584b6Eb5CE4eA


INFO:ocean:Asset/ddo published on-chain successfully.


ALG did = 'did:op:8C96958d4efd521f586829B3f8ede01B83E3478C'


# 4. Alice allows the algorithm for C2D for that data asset

In [12]:
from ocean_lib.assets.trusted_algorithms import add_publisher_trusted_algorithm
add_publisher_trusted_algorithm(DATA_ddo, ALG_ddo.did, config.metadata_cache_uri)
ocean.assets.update(DATA_ddo, publisher_wallet=alice_wallet)

INFO:ocean:Asset/ddo updated on-chain successfully.


'0xf3eb83a1b37d3ef0eba5efc58a3120f1b79302f923cd94e98fe03fbb1afd795e'

# 5. Bob acquires datatokens for data and algorithm

In [14]:
bob_wallet = Wallet(
    ocean.web3,
    os.getenv('TEST_PRIVATE_KEY2'),
    config.block_confirmations,
    config.transaction_timeout,
)
print(f"bob_wallet.address = '{bob_wallet.address}'")

# Alice shares access for both to Bob, as datatokens. Alternatively, Bob might have bought these in a market.
DATA_datatoken.transfer(bob_wallet.address, to_wei(5), from_wallet=alice_wallet)
ALG_datatoken.transfer(bob_wallet.address, to_wei(5), from_wallet=alice_wallet)

bob_wallet.address = '0x7F6470a7b08Bf4dfb3Ebd9330C5B3D594B1d5BB9'


'0x2f47dc92060f6df8a314c2cebe008af5625e48d838976c48a97acc6df4bf02b2'

# 6. Bob starts a compute job

In [15]:
DATA_did = DATA_ddo.did  # for convenience
ALG_did = ALG_ddo.did
DATA_DDO = ocean.assets.resolve(DATA_did)  # make sure we operate on the updated and indexed metadata_cache_uri versions
ALG_DDO = ocean.assets.resolve(ALG_did)

compute_service = DATA_DDO.get_service('compute')
algo_service = ALG_DDO.get_service('access')

from ocean_lib.web3_internal.constants import ZERO_ADDRESS
from ocean_lib.models.compute_input import ComputeInput

# order & pay for dataset
dataset_order_requirements = ocean.assets.order(
    DATA_did, bob_wallet.address, service_type=compute_service.type
)
DATA_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        dataset_order_requirements.amount,
        dataset_order_requirements.data_token_address,
        DATA_did,
        compute_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        dataset_order_requirements.computeAddress,
    )

# order & pay for algo
algo_order_requirements = ocean.assets.order(
    ALG_did, bob_wallet.address, service_type=algo_service.type
)
ALG_order_tx_id = ocean.assets.pay_for_service(
        ocean.web3,
        algo_order_requirements.amount,
        algo_order_requirements.data_token_address,
        ALG_did,
        algo_service.index,
        ZERO_ADDRESS,
        bob_wallet,
        algo_order_requirements.computeAddress,
)

compute_inputs = [ComputeInput(DATA_did, DATA_order_tx_id, compute_service.index)]
job_id = ocean.compute.start(
    compute_inputs,
    bob_wallet,
    algorithm_did=ALG_did,
    algorithm_tx_id=ALG_order_tx_id,
    algorithm_data_token=ALG_datatoken.address
)
print(f"Started compute job with id: {job_id}")

AssetNotConsumable: 2